In [1]:
import itertools
import json
import os
import random
from typing import Dict, List, Optional

from openai import AsyncOpenAI
from pydantic import BaseModel, Extra, Field

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

from pydantic_ai.models import ModelRequestParameters
from pydantic_ai.messages import ModelMessage
from pydantic_ai.settings import ModelSettings
import tiktoken

from config import settings

from src.simschat.models import (
    CharacterSpecification,
    SceneState,
    CharacterRelationState,
    CharacterState,
    create_dynamic_enum
)

In [2]:
# Use OpenRouter as inference endpoint
print(settings.llm_model)

# Pydantic Model
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

# OpenAI Client
aopenai_client = AsyncOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


# 0. Prepare Scene & Characters

In [3]:
scene_uid = "2f2462c1-bdef-4089-b206-c47decd841f3"

with open(f"simschat/scene/{scene_uid}.json", "r") as f:
    scene_dict = json.load(f)

print(json.dumps(scene_dict,indent=4))

{
    "uid": "2f2462c1-bdef-4089-b206-c47decd841f3",
    "stage": 0,
    "scene": {
        "revised_trope": "Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.",
        "utilized_reference_tropes": [
            "Defrosting Ice Queen",
            "SavvyGuyEnergeticGirl"
        ],
        "scene": {
            "location": "The Metropolitan Fa

# 1. Load Character Agents

In [4]:
class Character(BaseModel):
    uid: str
    spec: CharacterSpecification
    state: CharacterState

In [5]:
# Load Character Spec
character1_id="35f0c56f-263d-42df-846c-e1833d8ca0ab"
character2_id="00d66087-9b3b-46da-bd74-bf45cbe81d3c"

with open(f"simschat/characters/{character1_id}.json", "r") as f:
    character1_spec = CharacterSpecification(**json.load(f))
    
with open(f"simschat/characters/{character2_id}.json", "r") as f:
    character2_spec = CharacterSpecification(**json.load(f))
    
character1 = Character(
    uid=character1_id,
    spec=character1_spec,
    state=CharacterState.model_validate(scene_dict["character_states"]["character1"]),
)
character2 = Character(
    uid=character2_id,
    spec=character2_spec,
    state=CharacterState.model_validate(scene_dict["character_states"]["character2"]),
)

CHARACTERS = {
    character1_id: character1,
    character2_id: character2,
}
CHARACTERS

{'35f0c56f-263d-42df-846c-e1833d8ca0ab': Character(uid='35f0c56f-263d-42df-846c-e1833d8ca0ab', spec=CharacterSpecification(name='Zephyr Orion', gender='male', age=28, dialogue_tone='playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.', career='Astronaut', personality_traits=[PersonalityTrait(trait='Goofball', description='Enjoys joking and making others laugh, bringing a playful spirit to social situations.'), PersonalityTrait(trait='Materialistic', description='Loves acquiring new possessions and often leans towards bragging about them.'), PersonalityTrait(trait='Outgoing', description='Flourishes in social situations and enjoys being around people.'), PersonalityTrait(trait='Gloomy', description='Grows sad when left alone for too long.'), PersonalityTrait(trait='Ambitious', description='Continuously strives to reach new milestones in his career.')], hobbies=['Fitness', 'Cooki

In [6]:
# Load Allowed Actions
import random
with open("sims_interactions.json", "r") as f:
    full_sims_interactions = json.load(f)

# sample    
n = 10
sims_interactions = {k: random.sample(v, min(len(v), n)) for k,v in full_sims_interactions.items()}
print(sims_interactions.keys())

## Make enum of actions
possible_actions_model = create_dynamic_enum(
    "PossibleCharacterActions",
    itertools.chain(*[
        [f"{k}-{a['action']}" for a in v]
        for k,v in sims_interactions.items()
    ])
)

for k, v in sims_interactions.items():
    print(f"Interaction: {k}")
    print("\t{}".format(json.dumps([x['action'] for x in v])))

dict_keys(['Friendly', 'Ask', 'Romantic', 'Mean', 'Neutral'])
Interaction: Friendly
	["Gossip", "Talk About Hobby", "Ask to Join", "Dare", "Get to Know", "Enthuse about ...", "Tip", "Tickle", "Compliment", "Admire"]
Interaction: Ask
	["What Turns You Off?", "About Interests", "What Are Your Skills?", "Do You Like What You See?", "What Turns You On?", "What's Your Zodiac sign?", "How Much Money Do You Have?", "What's Your Job?", "What Do You Fear?", "What Do You Want?"]
Interaction: Romantic
	["Squeeze", "Suck Face", "Smooch", "Stroke Cheek", "Serenade", "Tender Kiss", "Have Private Wedding", "Make Out", "Juiced WooHoo & Kiss", "Kiss Hand"]
Interaction: Mean
	["Shout Forbidden Words at (Sim)", "Slap", "Scare", "Yell at", "Accuse of (trait)", "Push", "Petty Jab", "Mock (trait)", "Poodle (Werewolf)", "Insult"]
Interaction: Neutral
	["Shoo", "Ask to leave/Say Goodbye", "Join (in activity)", "Introduce to...", "Stop doing that", "Call Over"]


In [7]:
CharacterActionPredictionSystemPrompt = '''You are a creative storyteller that does the following.
Given informations about a character (Character) and context of a story scene (Scene Context)
(1) write the next action that the character will perform in the scene.
(2) update the character state after performing this action

# Given Informations
## Character
description: character persona that you will be taking
state:
* polarity: sentiment towards the character (positive/neutral/negative)
* emotion: emotional state of the character
* social_relations: a one-directional relationship with other characters. (ex. what character1 thinks of character2)
    * character_uid: uid of the target character
    * emotion: emotional relation towards the other character
    * knowledge: what the character knows about the other character 

## Scene Context
description of the current state of the scene

# Action Prediction
## Definition of 'Action'
a character performs an 'action' through the following steps
* 'think': consider what the character should say and what goals the character wants to achieve. write the thinking process here
* 'action_type': type of the action this character will actually perform
* 'description': description of the action this character will perform
    * the action must be detailed description of a single-verb action.
    * if there are dialogues, write the actual dialogues
    * if there are any background, items, characters involved, include the details about them
    * action must be maximum 2 sentences
    * don't make the actions obvious
* 'dialogue': the dialogue this character will perform (leave it empty if there are no dialogues)
* 'targets': List of characters (by uid) that will be affected. leave it empty if it there are no targets
    * use the uid value in character.social_relations to identify the character

### Action Type
type of actions that the character can perform are defined as the following
* actions are grouped by their 'Interaction' type (ex. Fiendly, Mean, ..)
* select the action in the form of "{{interaction}}-{{action}}" from the following dictionary (ex. Friendly-Admire)

Defined Actions:
{action_pool}

# State Update
## Emotions
utilize the following emotion descriptions (from Plutchik's emotion wheel) to describe the emotional state of the character
* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance social connections and overall resilience by radiating positivity
* "trust": Trust is the reassuring feeling of confidence and security in another person or situation. It builds from consistent, reliable interactions and underpins strong, supportive relationships. This emotion fosters cooperation and reduces anxiety by creating a sense of safety
* "fear": Fear is an instinctive response to perceived threats that activates our fight-or-flight mechanism. It heightens awareness and prepares our body to respond quickly to danger, making it essential for survival. Despite its discomfort, fear is a crucial signal that prompts protective action and risk assessment
* "surprise": Surprise occurs when we encounter the unexpected, momentarily halting our regular thought process. This emotion can be positive, neutral, or even negative, depending on the context, and often sparks curiosity about what comes next. Its brief nature helps redirect our focus and encourages adaptive responses to new situations
* "sadness": Sadness is a deep, reflective emotion that often emerges from loss, disappointment, or unmet expectations. It can lead to introspection and a desire for support as we navigate feelings of grief or dejection. Although challenging, sadness can also foster empathy and pave the way for emotional healing and growth
* "disgust": Disgust is an aversive emotion that signals rejection toward something perceived as harmful, unclean, or morally offensive. It serves as a protective mechanism, prompting us to avoid substances or situations that might be dangerous. This emotion plays a vital role in maintaining both physical health and ethical boundaries
* "anger": Anger arises when we perceive injustice, frustration, or a threat to our well-being, often urging us to act in response. It can manifest as physical tension and heightened energy, signaling that something in our environment needs to change. When managed effectively, anger can motivate constructive action and help assert personal boundaries
* "anticipation": Anticipation is the forward-looking emotion characterized by a mix of excitement and apprehension about future events. It motivates preparation and planning while balancing hope with cautious vigilance. This emotion bridges the gap between our present state and the potential for positive outcomes in the future

## Social Relations
a one-directional relationship with other characters. (ex. what character1 thinks of character2)
Update the knowledge of this character towards the other character
* character_uid: uid of the target character
* emotion: emotional relation towards the other character
* knowledge: what the character knows about the other character 
    
Return predicted action in the following json format
* action must be defined in 'Defined Actions'
{{
    "action": {{
        "think": str
        "action_type": str,
        "description": str,
        "dialogue": str,
        "targets": List[str]
    }},
    "updated_state": {{
        "sentiment": str,
        "emotion": str,
        "social_relations": [
            {{
                "character_uid": str,
                "emotion": str,
                "knowledge": List[str]
            }},
            ...
        ]
    }}
}}
'''.format(
    action_pool=json.dumps(sims_interactions)
)

CharacterActionPredictionUserTemplate = '''Information:
[Character]
{character}

[Scene Context]
{scene_context}'''

In [ ]:
# Action Prediction Tool Function
class CharacterAction(BaseModel):
    # character: str
    think: str
    action_type: possible_actions_model
    description: str
    dialogue: str
    targets: List[str]
    class Config:
        extra = Extra.forbid

class ActionPredictionResult(BaseModel):
    action: CharacterAction
    updated_state: CharacterState
    class Config:
        extra = Extra.forbid

async def predict_action(
    character: Character,
    scene_context: str
)->ActionPredictionResult:
    user_message = CharacterActionPredictionUserTemplate.format(
        character=character.model_dump_json(),
        scene_context=scene_context
    )
    print(user_message)
    result = await aopenai_client.beta.chat.completions.parse(
        model=settings.llm_model,
        messages=[
            {"role": "system", "content": CharacterActionPredictionSystemPrompt},
            {"role": "user", "content": user_message},
        ],
        response_format=ActionPredictionResult,
    )
    return result.choices[0].message.parsed

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_44179/2711018394.py:10: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid
/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_44179/2711018394.py:16: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [9]:
character = character1
scene_context="Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism."
predicted_action = await predict_action(
    character=character1,
    scene_context=scene_context
)

Information:
[Character]
{"uid":"35f0c56f-263d-42df-846c-e1833d8ca0ab","spec":{"name":"Zephyr Orion","gender":"male","age":28,"dialogue_tone":"playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.","career":"Astronaut","personality_traits":[{"trait":"Goofball","description":"Enjoys joking and making others laugh, bringing a playful spirit to social situations."},{"trait":"Materialistic","description":"Loves acquiring new possessions and often leans towards bragging about them."},{"trait":"Outgoing","description":"Flourishes in social situations and enjoys being around people."},{"trait":"Gloomy","description":"Grows sad when left alone for too long."},{"trait":"Ambitious","description":"Continuously strives to reach new milestones in his career."}],"hobbies":["Fitness","Cooking","Painting"],"living_conditions":["Resides in a modern city apartment filled with space memorabilia and

In [11]:
print(predicted_action.model_dump_json(indent=2))

{
  "action": {
    "think": "Zephyr wants to continue engaging Vivienne to build rapport despite her initial icy demeanor. He considers using his humor to break the ice further and show his genuine interest.",
    "action_type": "Friendly-Compliment",
    "description": "Zephyr gestures playfully towards Vivienne's fashionable outfit, saying, 'You must be the star of this gala—your style outshines the stars!'",
    "dialogue": "You must be the star of this gala—your style outshines the stars!",
    "targets": [
      "00d66087-9b3b-46da-bf45cbe81d3c"
    ]
  },
  "updated_state": {
    "sentiment": "positive",
    "emotion": "trust",
    "social_relations": [
      {
        "character_uid": "00d66087-9b3b-46da-bf45cbe81d3c",
        "emotion": "trust",
        "knowledge": [
          "Vivienne LaRoux is a renowned style influencer with an assertive demeanor.",
          "Zephyr is playful, outgoing, and eager to connect, even with someone initially dismissive."
        ]
      }
   

# 2. Initialize Storytelling Agent

In [12]:
SCENE_ACTION_HISTORY = []

In [14]:
StorytellingSystemPrompt = '''You are a creative storyteller that does the following.
Given the following informations about a story scene
(1) select a character that should perform the action by their uid
(2) use 'predict_action' to predict the next action
(3) Return the predicted action

# 1. Given Informations
## Explanation
* Scene Description:
    * trope: the story trope that the given scene follows
    * setting: location, background setting of the scene
    
* Characters: characters involved in this scene
    * uid: unique id value that differentiates the characters
    * description: description of the character

* History: History of actions performed in this scene

# 2. Use predict_action to predict the next action
predict_action receives the following information and returns a predicted action
Receives:
* character_uid: uid of the character that will perform the action
* scene_context: context summary that the character needs to know in order to predict an action
    * keep the context very detailed so that the character can predict the appropriate action

Returns:
action in the following schema
* think: the character's thought process that was used to predict the action
* action: the action that was predicted
* targets: UID of characters that are affected by this action
{{
    "think": str
    "action": str,
    "targets": List[str]
}}
    
Return in the following JSON
* return the action returned from predict_action as is 
{{
    "action": {{
        "think": str
        "action": str,
        "targets": List[str]
    }}
}}'''

StorytellingUserTemplate = '''[Scene Description]
trope:
{scene_trope}

setting:
{scene_setting}

[Characters]
{characters}

[History]
{history}'''

In [15]:
scene_trope = scene_dict["scene"]["revised_trope"]
scene_state = SceneState.model_validate(scene_dict["scene"]["scene"])

StorytellerAgent = Agent(
    model=model,
    name="storyteller",
    # deps_type=CharacterState,
    output_type=CharacterAction,
    system_prompt = StorytellingSystemPrompt
)

allowed_characters = create_dynamic_enum(
    "AllowedCharacterUIDs",
    list(CHARACTERS.keys())
)
@StorytellerAgent.tool
async def predict_action(
    ctx: RunContext[None],
    character_uid: allowed_characters,
    scene_context: str
)->Optional[CharacterAction]:
    character = CHARACTERS.get(character_uid, None)
    if not character:
        return None
    
    result = await predict_action(
        character_uid=character_uid,
        scene_context=scene_context
    )
    return result.action

In [16]:
user_message = StorytellingUserTemplate.format(
    scene_trope=scene_trope,
    scene_setting=scene_state.model_dump(),
    characters=json.dumps(
        [v.model_dump() for k,v in CHARACTERS.items()]
    ),
    history=json.dumps(
        [x.model_dump() for x in SCENE_ACTION_HISTORY]
    ),
)

result = await StorytellerAgent.run(user_message)

In [21]:
print(result.output.model_dump_json(indent=4))

{
    "think": "Zephyr, noticing that Vivienne's guard is lowering thanks to his humor, decides to keep the momentum going by offering her a playful and charming remark to cement this unexpected connection.",
    "action_type": "Friendly-Gossip",
    "description": "Zephyr looks at Vivienne with a wry smile, teasingly commenting on the dazzling fashion around them, hinting that even astronauts need to stay grounded in style.",
    "dialogue": "You know, with all these sparkling outfits, I might as well be landing on a dazzling galaxy myself.",
    "targets": []
}
